# Download ArcGIS Online Data to construct Bolivia Watch Geoportal

<div class="alert alert-info">
    <b>Original Application:</b>

Link Original Application 
(ArcGIS Experience)[https://experience.arcgis.com/experience/dcd5d132e90d40449c212018e4950a54/page/Homepage]
</div>

# ArcGIS API For Python

## Import Libraries and Packages

In [1]:
import os
import time

from datetime import datetime as dt
from IPython.display import Image, HTML

from arcgis.gis import GIS
import arcpy
from arcpy import env
from arcgis.raster.functions import *

# connect to GIS
from arcgis.gis import GIS
import getpass

## Access and Login into ArcGIS Account

In [2]:
# Prompt user to provide username and password

username = input('Enter username: ')
password = getpass.getpass("Enter your password: ") 
gis = GIS("https://seila.maps.arcgis.com/home", username, password)

Enter username:  carlos.mendez_seila
Enter your password:  ········


## Definition Local workSpace

In [3]:
try:
    os.chdir(r'C:\Users\USUARIO\Desktop\Bolivia_Watch_Geoportal\BW_Geospatial_Data')
    path = os.getcwd()
    print("Actual Workspace:", path)
except FileNotFoundError:
    print("The folder or directory doesnn't exist.")
    path = None

Actual Workspace: C:\Users\USUARIO\Desktop\Bolivia_Watch_Geoportal\BW_Geospatial_Data


## Adding data hosted on ArcGIS Online

In [4]:
### Adding data hosted on ArcGIS Online
query_string = 'owner:"angie.forero_seila"'
BW_data = gis.content.search(query=query_string, max_items=1000)
print(f"The user has published {len(BW_data)} elements in ArcGIS Online.")

The user has published 108 elements in ArcGIS Online.


## Display items in ArcGIS Online

In [5]:
for item in BW_data:
    print(f"{item.title} | {item.id} | {item.type}")

BW_Cambio_SPEI_WTL1 | c84d102bc51248bb891c3b8200bccd5d | Map Service
BW_SDI | 89cb082320ac484d8724c53563237340 | Insights Page
BWII_BH_PPT | 744eecb38e2d4cacb552a1fac23de69b | Web Map
BW_Proyeccion_P | 53b5159030394fd3a1aa0640898d0a2d | Web Map
BWII_BH_ESC_WFL1 | 1373c352d8634dcb95abd8ec0f605435 | Service Definition
BW_ENSO | 4a1bb40e0dfc4e518689a8746ae5c17e | Web Map
BWII_IA_II | e7018f5521004c63820d549cfd8ffedc | Web Map
BWII_Resultados_BH | 653b7a933429459088dfdb74855374ce | Web Map
BW_Indice_Combinado_WFL1 | a2b4abbd222e4bc3a5778530618512d1 | Service Definition
BWII_Clima_BH | 0b17832ca9214deba75521d0cabd9876 | Web Map
BW_Indice_Combinado_3 | 55d734ba45814609a28db489e5d6e4b2 | Web Map
BW_Cambio_SPEI_WFL1 | 9dda45c2374344f789ddfd4448e783b8 | Service Definition
BWII_BH_Escorrentia | b3473c3e55ec40548a91583afff15a80 | Web Map
BW_ENSO_WFL1 | a335a964f91843d8876dc6190b916593 | Feature Service
BWII_BH_CESP_WFL1 | 24553e7f3c9349a48705b03a51e69827 | Service Definition
WBB_Limits_II | 02e09

In [6]:
feature_layers = [item for item in BW_data if item.type == "Feature Layer"]
print(f"Se encontraron {len(feature_layers)} Feature Layers.")

Se encontraron 0 Feature Layers.


## Download items published in ArcGIS Online

In [17]:
if path and BW_data:
    for item in BW_data:
        try:
            print(f"Descargando: {item.title} ({item.id})")
            item.download(save_path=path)
        except Exception as e:
            print(f"Error al descargar {item.title}: {e}")
else:
    print("No se ha definido el directorio o no hay elementos para descargar.")

Descargando: BW_ENSO (4a1bb40e0dfc4e518689a8746ae5c17e)
Descargando: BWII_IA_II (e7018f5521004c63820d549cfd8ffedc)
Descargando: BWII_Resultados_BH (653b7a933429459088dfdb74855374ce)
Descargando: BWII_Area (20dadaabbd064f45b51c47506a95d582)
Descargando: BW_Proyeccion_P (53b5159030394fd3a1aa0640898d0a2d)
Descargando: BW_SDI (89cb082320ac484d8724c53563237340)
Descargando: BWII_BH_PPT (744eecb38e2d4cacb552a1fac23de69b)
Descargando: SPI_Municipios_WFL1 (fe3edb980ced457cae82d723002b6e30)
Descargando: Sheet_WBB (2c03deb2d24f49e191887b19e468636d)
Descargando: BWII_Clima_BH (0b17832ca9214deba75521d0cabd9876)
Descargando: BW_ENSO_WFL1 (a335a964f91843d8876dc6190b916593)
Descargando: Observatorio (dcd5d132e90d40449c212018e4950a54)
Descargando: ERA_SEI_CH (0c6c8aacb11d4046ae52c3118d080a36)
Descargando: WBB_Limits_II (02e0967e6011476fa7fbc1bb154baadb)
Descargando: Breaking boundaries (232020f06b644774b6525a866ebd9ae7)
Descargando: BWII_Clima (ddbe2f23385a442d97d4db2095d07a89)
Descargando: BWII_Resul

## Download Data in .Csv Format

In [20]:
from arcgis.features import FeatureLayer
import pandas as pd

# Carpeta destino
output_folder = r"C:\Users\USUARIO\Desktop\Bolivia_Watch_Geoportal\BW_Geospatial_Data\Descargas_GIS"
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre todos los ítems encontrados
for item in BW_data:
    print(f"Procesando: {item.title} | {item.type}")

    try:
        # Si es Web Map
        if item.type == "Web Map":
            webmap_data = item.get_data()
            operational_layers = webmap_data.get("operationalLayers", [])

            for layer_info in operational_layers:
                layer_url = layer_info.get("url")
                if layer_url:
                    layer = FeatureLayer(layer_url)
                    df = layer.query(where="1=1", out_fields="*", as_df=True)
                    filename = f"{layer_info['title'].replace(' ', '_')}.csv"
                    df.to_csv(os.path.join(output_folder, filename), index=False)
                    print(f"  ✅ Descargado desde Web Map: {filename}")

        # Si es Feature Service
        elif item.type == "Feature Service":
            for lyr in item.layers:
                df = lyr.query(where="1=1", out_fields="*", as_df=True)
                # filename = f"{lyr.properties.name.replace(' ', '_')}.csv"
                df.to_csv(os.path.join(output_folder, filename), index=False)
                print(f"  ✅ Descargado desde Feature Service: {filename}")

        else:
            print("  ⚠️ Tipo no compatible para descarga directa.")

    except Exception as e:
        print(f"  ❌ Error procesando {item.title}: {e}")

Procesando: BW_ENSO | Web Map
  ✅ Descargado desde Web Map: SST_Promedio_[°C].csv
  ✅ Descargado desde Web Map: Country_border.csv
  ✅ Descargado desde Web Map: Regiones_Hidroclimáticas.csv
  ✅ Descargado desde Web Map: Índices_ENSO.csv
Procesando: BWII_IA_II | Web Map
  ❌ Error procesando BWII_IA_II: {'error': {'code': 400, 'message': 'Invalid URL', 'details': ['Invalid URL']}}
Procesando: BWII_Resultados_BH | Web Map
  ❌ Error procesando BWII_Resultados_BH: {'error': {'code': 400, 'message': 'Invalid URL', 'details': ['Invalid URL']}}
Procesando: BWII_Area | Web Map
  ✅ Descargado desde Web Map: Macrocuencas.csv
  ✅ Descargado desde Web Map: Macrocuencas.csv
  ✅ Descargado desde Web Map: Calibracion.csv
  ✅ Descargado desde Web Map: Ríos.csv
  ✅ Descargado desde Web Map: Límites_internacionales.csv
  ✅ Descargado desde Web Map: Lago.csv
  ✅ Descargado desde Web Map: Límite_nacional.csv
  ✅ Descargado desde Web Map: Límite_municipal.csv
  ✅ Descargado desde Web Map: Capital_municipal_

## Download data in .shp format

In [7]:
import arcpy
from arcgis.features import FeatureLayer

# Carpeta destino
output_folder = r"C:\Users\USUARIO\Desktop\Bolivia_Watch_Geoportal\BW_Geospatial_Data\Descargas_GIS"
arcpy.env.workspace = output_folder
arcpy.env.overwriteOutput = True

# Iterar sobre ítems
for item in BW_data:
    print(f"Procesando: {item.title} | {item.type}")

    try:
        # Web Map
        if item.type == "Web Map":
            webmap_data = item.get_data()
            operational_layers = webmap_data.get("operationalLayers", [])

            for layer_info in operational_layers:
                layer_url = layer_info.get("url")
                if layer_url:
                    layer = FeatureLayer(layer_url)
                    fs_name = layer_info['title'].replace(" ", "_")
                    output_fc = os.path.join(output_folder, f"{fs_name}.shp")
                    layer.query(where="1=1", out_fields="*", out_path=output_fc)
                    print(f"  ✅ Shapefile guardado desde Web Map: {output_fc}")

        # Feature Service
        elif item.type == "Feature Service":
            for lyr in item.layers:
                fs_name = lyr.properties.name.replace(" ", "_")
                output_fc = os.path.join(output_folder, f"{fs_name}.shp")
                lyr.query(where="1=1", out_fields="*", out_path=output_fc)
                print(f"  ✅ Shapefile guardado desde Feature Service: {output_fc}")

        else:
            print("  ⚠️ Tipo no compatible para descarga directa.")

    except Exception as e:
        print(f"  ❌ Error procesando {item.title}: {e}")

Procesando: BW_Cambio_SPEI_WTL1 | Map Service
  ⚠️ Tipo no compatible para descarga directa.
Procesando: BW_SDI | Insights Page
  ⚠️ Tipo no compatible para descarga directa.
Procesando: BWII_BH_PPT | Web Map
  ✅ Shapefile guardado desde Web Map: C:\Users\USUARIO\Desktop\Bolivia_Watch_Geoportal\BW_Geospatial_Data\Descargas_GIS\Índice_de_aridez.shp
  ✅ Shapefile guardado desde Web Map: C:\Users\USUARIO\Desktop\Bolivia_Watch_Geoportal\BW_Geospatial_Data\Descargas_GIS\Caudal_específico.shp
  ✅ Shapefile guardado desde Web Map: C:\Users\USUARIO\Desktop\Bolivia_Watch_Geoportal\BW_Geospatial_Data\Descargas_GIS\Escorrentía.shp
  ✅ Shapefile guardado desde Web Map: C:\Users\USUARIO\Desktop\Bolivia_Watch_Geoportal\BW_Geospatial_Data\Descargas_GIS\Evapotranspiración.shp
  ✅ Shapefile guardado desde Web Map: C:\Users\USUARIO\Desktop\Bolivia_Watch_Geoportal\BW_Geospatial_Data\Descargas_GIS\Precipitación.shp
  ✅ Shapefile guardado desde Web Map: C:\Users\USUARIO\Desktop\Bolivia_Watch_Geoportal\BW_G